In [1]:
from datasets import load_dataset, load_from_disk
import numpy as np
import seaborn as sns
from rouge_score import rouge_scorer, scoring
import evaluate
import pandas as pd
sns.set_context("paper")

import matplotlib.pyplot as plt
import numpy

plt.style.use('seaborn-poster')
plt.style.use('fivethirtyeight')

/home/yangsibo/.conda/envs/jailbreak/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/yangsibo/.conda/envs/jailbreak/lib/python3.11/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/tmp/ipykernel_2762212/1208568996.py:12: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-poster')


In [2]:
dataset = load_dataset("swj0419/BookMIA")

In [3]:
books = dataset['train']['snippet']

In [4]:
datatype = 'books'
if datatype == 'lyrics':
    dataset = load_from_disk('/scratch/gpfs/yangsibo/cache/datasets_lyrics')
    context_len = 100
    completion_len = 100
    key = 'lyrics'
    thres = 0.1
    np.random.seed(1234)
    indices = np.random.randint(len(dataset['train']), size=30000)
elif datatype == 'news':
    dataset = load_from_disk('/scratch/gpfs/yangsibo/cache/datasets_cnn_dailymail')
    context_len = 500
    completion_len = 500
    key = 'article'
    thres = 0.15
    np.random.seed(1234)
    indices = np.random.randint(len(dataset['train']), size=30000)
elif datatype == 'books':
    dataset = load_dataset("swj0419/BookMIA")
    context_len = 250
    completion_len = 250
    key = 'snippet'
    thres = 0.15
    indices = np.where(np.asarray(dataset['train']['label'])==1)[0]
    

testing_chunks = dataset['train'].select(indices)

In [5]:
rouge = evaluate.load('rouge')
examples = testing_chunks

kept = []
for e in examples:
    words = e[key].split(' ')
    if len(words) < context_len + completion_len:
        print(len(words))
        continue
    prompt = ' '.join(words[:context_len])
    gt =  ' '.join(words[context_len:context_len+completion_len])
    r = rouge.compute(predictions=[prompt], references=[gt], use_aggregator=False) 
    if r['rougeL'][0] < thres:
        e['prompt'] = prompt
        e['gt'] = gt
        kept.append(e)
        if len(kept) == 200:
            break
        print(r)
        print(len(kept))

{'rouge1': [0.3921568627450981], 'rouge2': [0.03543307086614173], 'rougeL': [0.13333333333333336], 'rougeLsum': [0.13333333333333336]}
1
{'rouge1': [0.35294117647058826], 'rouge2': [0.051181102362204724], 'rougeL': [0.13333333333333333], 'rougeLsum': [0.13333333333333333]}
2
{'rouge1': [0.37254901960784315], 'rouge2': [0.03543307086614173], 'rougeL': [0.13725490196078433], 'rougeLsum': [0.13725490196078433]}
3
{'rouge1': [0.38113948919449897], 'rouge2': [0.05128205128205128], 'rougeL': [0.12966601178781925], 'rougeLsum': [0.12966601178781925]}
4
{'rouge1': [0.32015065913371], 'rouge2': [0.04158790170132325], 'rougeL': [0.128060263653484], 'rougeLsum': [0.128060263653484]}
5
{'rouge1': [0.4171539961013645], 'rouge2': [0.0273972602739726], 'rougeL': [0.13645224171539963], 'rougeLsum': [0.13645224171539963]}
6
{'rouge1': [0.2974559686888454], 'rouge2': [0.03536345776031434], 'rougeL': [0.12524461839530332], 'rougeLsum': [0.12524461839530332]}
7
{'rouge1': [0.30158730158730157], 'rouge2': 

In [9]:
if datatype == 'lyrics':
    df = pd.DataFrame(kept).drop(columns=['Unnamed: 0'])
elif datatype == 'news':
    df = pd.DataFrame(kept)
elif datatype == 'books':
    df = pd.DataFrame(kept)
df.to_csv(f'{datatype}.csv', index=False)

In [7]:
df

,book_id,book,snippet_id,snippet,label,prompt,gt
0,11,Emma.txt,23,"our acquaintance, when I did like him, when I ...",1,"our acquaintance, when I did like him, when I ...","love with him?-- very wrong, very wrong indeed..."
1,18,Hitchhiker's Guide To The Galaxy.txt,26,the main reason why he had had such a wild and...,1,the main reason why he had had such a wild and...,have thought was a heroic effort to cross the ...
2,43,The Turn of the Screw.txt,15,many a corner round which I expected to come u...,1,many a corner round which I expected to come u...,"that, whether the children really saw or not--..."
3,40,The Picture of Dorian Gray.txt,18,"yield to it. Resist it, and your soul grows si...",1,"yield to it. Resist it, and your soul grows si...","a new world, but rather a new chaos, that it c..."
4,0,1984.txt,13,the boughs of the elm trees were swaying very ...,1,the boughs of the elm trees were swaying very ...,across a chair. The Physical Jerks would begin...
...,...,...,...,...,...,...,...
195,35,The Da Vinci Code.txt,92,attractive young woman always drew eyes away f...,1,attractive young woman always drew eyes away f...,gaze. Her eyes were olive-green-incisive and c...
196,49,treasure island.txt,49,the funeral he was as drunk road. He was plain...,1,the funeral he was as drunk road. He was plain...,may now be?” hard and fast like a man on a ste...
197,33,The Age of Innocence.txt,27,to do what you all do--I want to feel cared fo...,1,to do what you all do--I want to feel cared fo...,"an esteem for them."" The terms were hardly ade..."
198,45,Things Fall Apart.txt,66,"end. ""It is not our custom to fight for our go...",1,"end. ""It is not our custom to fight for our go...",of the new faith were thenceforth excluded fro...


In [8]:
sns.histplot(df['tag'])

KeyError: 'tag'

In [ ]:
lengths = [len(x) for x in df['lyrics']]
print(np.mean(lengths), np.std(lengths))
sns.histplot(lengths)

: 